<a href="https://colab.research.google.com/github/eunsun53/ML_proj/blob/main/%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5track_Transfer_Learning_%EC%8B%A4%EB%AC%B4%EC%9D%91%EC%9A%A9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Transfer Learning 


> pre-trained 모델을 불러와서 도메인 데이터를 학습시키는 경우 


input 형태와 out 형태 및 크기를 맞춰주어야 정상 작동함, 

이를 알고 데이터형의 변형을 잘 완수하고 이를 알 수 있어야 함 

In [4]:
import tensorflow as tf 
from tensorflow.keras import layers, Sequential 
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.datasets import fashion_mnist

import numpy as np 
import pandas as pd

In [23]:
def load_transfer_model():
  # input shape = (96, 96, 3) -> cifar10 데이터의 크기 
  # include_top = False, weights = imagenet'
  base_model = ResNet50(include_top = False, weights = 'imagenet', input_shape = (84, 84, 3)) # resnet50은 3채널 input데이터 학습 모델임 

  #base_model이 훈련 가능하지 않게 설정 
  base_model.trainable = False 

  #layer를 base_model 앞뒤로 추가함 
  num_classes = 10 
  transfer_model = Sequential([
      layers.UpSampling2D(size = (3, 3), interpolation = 'bilinear'), #size: 몇배 늘릴지, interpolation = 
      base_model,
      layers.GlobalAveragePooling2D(),
      layers.Dense(num_classes, activation = 'softmax') 
  ])

  return transfer_model 

In [16]:
def duplicated_input(arr, num_feature_map):
  in_arr = np.empty([num_feature_map , 28, 28, 3])
  for idx, item in enumerate(in_arr):
    item[:, :, 0] = arr[idx]
    item[:, :, 1] = arr[idx]
    item[:, :, 2] = arr[idx]

  return in_arr 


In [25]:
from tensorflow.keras.utils import to_categorical 

In [27]:
def main(transfer_model = None, epochs = 3):
  #원본 코드는 cifar10을 내장함, 우리는 fashion Mnist레 적용해볼 것임  
  (x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
  x_train, y_train, x_test, y_test = x_train[:5000], y_train[:5000], x_test[:5000], y_test[:5000]
  y_train_onehot = to_categorical(y_train)
  y_test_onehot = to_categorical(y_test)
  print('x_train.shape: ', x_train.shape)

  # input data의 dimension 조정 
  x_train_reshaped = x_train.reshape(-1, 28, 28, 1) # gray scale 이미지 이므로 채널 1개로 차원 확장  
  x_test_reshaped = x_test.reshape(-1, 28, 28, 1)
  '''
  gray 1차원 ->(방법1) r,g,b 3차원으로 복사, 병합  
  x_train_red = x_train_reshaped.copy()
  x_train_green = x_train_reshaped.copy()
  x_train_blue = x_train_reshaped.copy()
  
  x = np.block([x_train_red, x_train_green, x_train_blue])
  '''
  print('x_train_reshaped.shape: ', x_train_reshaped.shape) 
  
  # gray 1차원 ->(방법2) r,g,b 3차원으로 새로운 3차원 리스트 만들어서 각각 넣어주기  
  x_train_res_in = duplicated_input(x_train, len(x_train))
  x_test_res_in = duplicated_input(x_test, len(x_test))
  print('x_train_res_in.shape: ', x_train_res_in.shape)

  #transfer model이 없을땐 불러오고 있으면 넘어가는 코드 만들기 
  if not transfer_model:
    transfer_model = load_transfer_model()
  
  #optimizer, loss, metric 적용 
  optimizer = Adam(learning_rate = 0.001)
  transfer_model.compile(loss= 'categorical_crossentropy', optimizer=optimizer, metrics = ['accuracy'] )

  # 모델 학습 
  hist = transfer_model.fit(x_train_res_in, y_train_onehot, epochs = epochs, batch_size = 500)

  # 완성된 모델 확인: summary 
  transfer_model.summary()

  # 테스트 데이터로 모델 평가 
  loss, accuracy = transfer_model.evaluate(x_test_res_in, y_test_onehot)
  print("accuracy: ", accuracy*100)

main()

x_train.shape:  (5000, 28, 28)
x_train_reshaped.shape:  (5000, 28, 28, 1)
x_train_res_in.shape:  (5000, 28, 28, 3)
Epoch 1/3
10/10 [==============================] - 127s 12s/step - loss: 1.9670 - accuracy: 0.3842
Epoch 2/3
10/10 [==============================] - 122s 12s/step - loss: 0.7786 - accuracy: 0.7324
Epoch 3/3
 2/10 [=====>........................] - ETA: 1:36 - loss: 0.6265 - accuracy: 0.7970

KeyboardInterrupt: ignored